# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# Part 1 data file
cities_metadata_path = "../WeatherPy/city_data.csv"

# Read the metadata 
cities_metadata_df = pd.read_csv(cities_metadata_path)

In [3]:
cities_metadata_df.columns

Index(['City', 'Lat', 'Long', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Time'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [5]:
#Figure(layout=FigureLayo# Customise the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [6]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_metadata_df[["Lat", "Long"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
location_hum = cities_metadata_df.dropna()
loc_humidity = cities_metadata_df["Humidity"].astype(float)


In [7]:
# Create a humidity heatmap layer
fig1 = gmaps.figure(map_type='HYBRID', center = (0, 30) ,zoom_level = 2)

humidity_layer = gmaps.heatmap_layer(locations, weights=loc_humidity, 
                                 dissipating=False, max_intensity=250,
                                 point_radius = 4)

fig1.add_layer(humidity_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
# Narrow down the DataFrame to find your ideal weather condition

#A max temperature lower than 30 degrees but higher than 25.
#Wind speed less than 4.5 m/s.
#Zero cloudiness.

ideal_max_temp = 33
ideal_min_temp = 24
ideal_wind_speed = 4.5
ideal_cloudiness = 0 

#Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

hotel_df = cities_metadata_df[(cities_metadata_df["Max Temp"] <= ideal_max_temp) &
                              (cities_metadata_df["Max Temp"] >= ideal_min_temp) &
                              (cities_metadata_df["Wind Speed"] <= ideal_wind_speed) &
                              (cities_metadata_df["Cloudiness"] == ideal_cloudiness)] 

hotel_df.head(20)


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time
1,Cedar City,37.6775,-113.0619,27.44,35,0,2.06,US,1659977545
8,Makakilo City,21.3469,-158.0858,25.11,71,0,4.12,US,1659977547
49,Laguna,38.4210,-121.4238,26.12,65,0,3.09,US,1659977557
172,Preeceville,51.9500,-102.6677,24.94,51,0,3.56,CA,1659977594
255,Kumul,42.8000,93.4500,32.69,16,0,2.18,CN,1659977029
282,Wajir,1.7471,40.0573,29.98,35,0,4.12,KE,1659977620
283,Hailey,43.5196,-114.3153,27.71,29,0,4.12,US,1659977620
382,Tuscaloosa,33.2098,-87.5692,32.34,73,0,1.54,US,1659977549
398,Arrondissement de Mende,44.6667,3.5000,24.33,25,0,3.54,FR,1659977648
425,Magna,40.7091,-112.1016,30.02,38,0,1.79,US,1659977654


In [9]:
hotel_df.shape

(15, 9)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# set up additional columns to hold information
hotel_df['Hotel Name'] = ""
hotel_df = hotel_df.reset_index(drop=True)
hotel_df.head(20)


C:\Users\dalem\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time,Hotel Name
0,Cedar City,37.6775,-113.0619,27.44,35,0,2.06,US,1659977545,
1,Makakilo City,21.3469,-158.0858,25.11,71,0,4.12,US,1659977547,
2,Laguna,38.4210,-121.4238,26.12,65,0,3.09,US,1659977557,
3,Preeceville,51.9500,-102.6677,24.94,51,0,3.56,CA,1659977594,
4,Wajir,1.7471,40.0573,29.98,35,0,4.12,KE,1659977620,
5,Hailey,43.5196,-114.3153,27.71,29,0,4.12,US,1659977620,
6,Arrondissement de Mende,44.6667,3.5000,24.33,25,0,3.54,FR,1659977648,
7,Magna,40.7091,-112.1016,30.02,38,0,1.79,US,1659977654,
8,Vernon,50.2581,-119.2691,26.07,43,0,0.51,CA,1659977668,
9,‘Āhuimanu,21.4447,-157.8378,25.76,75,0,3.60,US,1659977671,


In [11]:
hotel_df.dtypes

City           object
Lat           float64
Long          float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Time            int64
Hotel Name     object
dtype: object

In [12]:
hotel_df.shape

(15, 10)

In [13]:
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time,Hotel Name
0,Cedar City,37.6775,-113.0619,27.44,35,0,2.06,US,1659977545,
1,Makakilo City,21.3469,-158.0858,25.11,71,0,4.12,US,1659977547,
2,Laguna,38.4210,-121.4238,26.12,65,0,3.09,US,1659977557,
3,Preeceville,51.9500,-102.6677,24.94,51,0,3.56,CA,1659977594,
4,Wajir,1.7471,40.0573,29.98,35,0,4.12,KE,1659977620,
5,Hailey,43.5196,-114.3153,27.71,29,0,4.12,US,1659977620,
6,Arrondissement de Mende,44.6667,3.5000,24.33,25,0,3.54,FR,1659977648,
7,Magna,40.7091,-112.1016,30.02,38,0,1.79,US,1659977654,
8,Vernon,50.2581,-119.2691,26.07,43,0,0.51,CA,1659977668,
9,‘Āhuimanu,21.4447,-157.8378,25.76,75,0,3.60,US,1659977671,


In [14]:
# Set the hotel search radius

search_type = "lodging"
search_radius = 5000

search_loc = "" 


params = {
    "types": search_type,
    "radius": search_radius,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get location from df
    search_city = row['City']
 
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]
    
    #change location each iteration while leaving original params in place
    #params["location"] = f"{lat},{lng}"
    
    search_loc = f"{lat},{lng}"
    params["location"] = search_loc
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #moved
    search_result = requests.get(base_url, params=params)
    hotel_name = search_result.json()
    
    #Store the first hotel in dataframe
    
    try:
    # Print the name and address of the first hotel to appear
        print(f"A hotel in {search_city} to try is {hotel_name['results'][0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "N/A"
        print("No results... skipping.")
        
    print("----------------------------")

A hotel in Cedar City to try is Hampton Inn Cedar City.
----------------------------
A hotel in Makakilo City to try is Marriott's Ko Olina Beach Club.
----------------------------
A hotel in Laguna to try is Holiday Inn Express & Suites Elk Grove Central - Hwy 99, an IHG Hotel.
----------------------------
A hotel in Preeceville to try is Golden West Hotel.
----------------------------
A hotel in Wajir to try is Mahatallah Hotel.
----------------------------
A hotel in Hailey to try is Wood River Inn & Suites.
----------------------------
A hotel in Arrondissement de Mende to try is Le Plateau du Roy.
----------------------------
A hotel in Magna to try is Kopper View Manufactured Home Community.
----------------------------
A hotel in Vernon to try is Prestige Vernon Lodge and Conference Centre.
----------------------------
A hotel in ‘Āhuimanu to try is Paradise Bay Resort.
----------------------------
A hotel in Dawlatabad to try is WASIM ASGAR KHAN.
----------------------------
No

In [15]:
hotel_df.shape

(15, 10)

In [16]:
hotel_df.head(20)

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time,Hotel Name
0,Cedar City,37.6775,-113.0619,27.44,35,0,2.06,US,1659977545,Hampton Inn Cedar City
1,Makakilo City,21.3469,-158.0858,25.11,71,0,4.12,US,1659977547,Marriott's Ko Olina Beach Club
2,Laguna,38.4210,-121.4238,26.12,65,0,3.09,US,1659977557,Holiday Inn Express & Suites Elk Grove Central...
3,Preeceville,51.9500,-102.6677,24.94,51,0,3.56,CA,1659977594,Golden West Hotel
4,Wajir,1.7471,40.0573,29.98,35,0,4.12,KE,1659977620,Mahatallah Hotel
5,Hailey,43.5196,-114.3153,27.71,29,0,4.12,US,1659977620,Wood River Inn & Suites
6,Arrondissement de Mende,44.6667,3.5000,24.33,25,0,3.54,FR,1659977648,Le Plateau du Roy
7,Magna,40.7091,-112.1016,30.02,38,0,1.79,US,1659977654,Kopper View Manufactured Home Community
8,Vernon,50.2581,-119.2691,26.07,43,0,0.51,CA,1659977668,Prestige Vernon Lodge and Conference Centre
9,‘Āhuimanu,21.4447,-157.8378,25.76,75,0,3.60,US,1659977671,Paradise Bay Resort


In [17]:
#Plot markers on the heatmap.
locations = hotel_df[["Lat", "Long"]]

In [18]:
# NOTE: Do not change any of the code in this cell



# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [19]:
marker_layer = gmaps.symbol_layer(locations, info_box_content = hotel_info)
fig2 = gmaps.figure(center = (0, 30) ,zoom_level = 2)
fig2.add_layer(marker_layer)
fig2

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Create a combined map
fig3 = gmaps.figure(center = (0, 30) ,zoom_level = 2)
fig3.add_layer(marker_layer)
fig3.add_layer(humidity_layer)


fig3

Figure(layout=FigureLayout(height='420px'))